# Problem 2: German Credit Dataset (72 points)

In [382]:
import pandas as pd
from collections import Counter
df = pd.read_csv("GermanCredit.csv")

## **Preprocessing**

### [8 pts] Drop the 3 columns that contribute the least to the dataset. These would be the columns with the highest number of non-zero 'none' values. Break ties by going left to right in columns. (Your code should be generalizable to drop n columns, but for the rest of the analysis, you can call your code for n=3.)


In [435]:
#2.1(pre)
def drop_n_cols(df,n):

    #list of coloumns in data frame
    cols = list(df.columns.values)

    #creating ad dictionary to store count of 'none' for each column
    cols_dict = dict()
    for col in cols:
        cols_dict[col] = df.loc[df[col] == 'none',col].count()

        #Convert the dictionary to Counter to find n entries with highetst 'none' count
    cols_dict = Counter(cols_dict)
    cols_dict = cols_dict.most_common(3)

        #cols_dict is now a list of tuple, drop the desired columns
    for tup in cols_dict:
        df = df.drop(tup[0],axis=1).copy()
    return df
df = drop_n_cols(df,3)

### [4 pts] Certain values in some of the columns contain unnecessary apostrophes (‘). Remove the apostrophes.

In [384]:
#2.2(pre)
#!!!!!!!!!!need verification about the meaning of apostrophe on data set
#this function simply remove all apostrophe in the entire dataset
def remove_apostrophe(df):
    for cols in list(df.columns.values):
        df[cols].replace(regex=True,inplace=True,to_replace=r"'",value=r'')
    return df

In [386]:
df = remove_apostrophe(df)

###### Helper Method for 2.3-2.6(preprocessing)

In [387]:
#Helper for 2.3 - 2.6(pre)
def modify(df,modifying_table,colname):
    for tup in modifying_table:
        df.loc[df[colname] == tup[0], colname]= tup[1]
    return df

### [5 pts] The checking_status column has values in 4 categories: 'no checking', '<0', '0<=X<200', and '>=200'. Change these to 'No Checking', 'Low', 'Medium', and 'High' respectively

In [388]:
#2.3(pre)
def modify_checking_status(df):
    modifying_table = [("no checking","No checking"),("<0","Low"),("0<=X<200","Medium"),(">=200","High")]
    df = modify(df,modifying_table,"checking_status")
    return df
    

In [389]:
df = modify_checking_status(df)

### [5 pts] The savings_status column has values in 4 categories: 'no known savings', '<100', '100<=X<500', '500<=X<1000', and '>=1000'. Change these to 'No Savings', 'Low', 'Medium', 'High', and 'High' respectively. (Yes, the last two are both 'High')

In [390]:
#2.4(pre)
def modify_savings_status(df):
    modifying_table = [("no known savings","No Savings"),("<100","Low"),("100<=X<500","Medium"),("500<=X<1000","High"),(">=1000","High")]
    return modify(df,modifying_table,'savings_status')
    

In [391]:
df = modify_savings_status(df)

### [4 pts] Change class column values from 'good' to '1' and 'bad' to '0'.

In [392]:
#2.5(pre)
def modify_class(df):
    modifying_table = [("good","1"),("bad","0")]
    return modify(df,modifying_table,"class")


In [393]:
df = modify_class(df)

### [5 pts] Change the employment column value 'unemployed' to 'Unemployed', and for the others, change to 'Amateur', 'Professional', 'Experienced' and 'Expert', depending on year range.


In [394]:
#2.6(pre)
def modify_employment(df):
    modifying_table = [('umemployed',"Unemployed"),("<1","Amatuer"),("1<=X<4","Professional"),("4<=X<7","Experienced"),(">=7","Expert")]
    return modify(df,modifying_table,"employment")

In [395]:
df = modify_employment(df)

## **Analysis**

### [5 pts] Often we need to find correlations between categorical attributes, i.e. attributes that have values that fall in one of several categories, such as "yes"/"no" for attr1, or "low","medium","high" for attr2.
One such correlation is to find counts in combinations of categorial values across attributes, as in how many instances are "yes" for attr1 and "low" for attr2. A good way to find such counts is to use the Pandas crosstab function. Do this for the following two counts.
#### a)[3 pts] Get the count of each category of foreign workers (yes and no) for each class of credit (good and bad).
#### b)[2 pts] Similarly, get the count of each category of employment for each category of saving_status.


In [396]:
#2.1 a (ana)
def correl_foreign_class(df):
    return pd.crosstab(df['foreign_worker'],df['class']).rename(columns={'0':"bad","1":"good"})

In [397]:
df2 = correl_foreign_class(df)

In [398]:
#2.2 b (ana)
def correl_employment_saving(df):
    return pd.crosstab(df['employment'],df['savings_status'])

In [399]:
df3 =correl_employment_saving(df)

### [4 pts] Find the average credit_amount of single males that have 4<=X<7 years of employment. You can leave the raw result as is, no need for rounding

In [400]:
# 2.3 (ana)
## 4<=X<7 was updated to "Experienced"
average_credit_amount = df.loc[(df['personal_status'] == 'male single') & (df['employment']=='Experienced')].credit_amount.mean()

### [4 pts] Find the average credit duration for each of the job types. You can leave the raw result as is, no need for rounding.

In [401]:
# 2.4 (ana)
avg_duration_job = df.groupby('job')['duration'].mean()

In [402]:
avg_duration_job

job
high qualif/self emp/mgmt    25.168919
skilled                      21.411111
unemp/unskilled non res      17.363636
unskilled resident           16.535000
Name: duration, dtype: float64

### [4 pts] For the purpose 'education', what is the most common checking_status and savings_status? Your code should print:
    Most common checking status: ...
    Most common savings status: ...

In [430]:
#2.5(ana)
mc_checking = str(df.loc[(df['purpose']=='education'),['checking_status']].mode().iloc[0,0])

In [431]:
print("Most common checking status: ",mc_checking)

Most common checking status:  No checking


In [433]:
mc_saving = str(df.loc[(df['purpose']=='education'),['savings_status']].mode().iloc[0,0])

In [434]:
print("Most common savings status: ",mc_saving)

Most common savings status:  Low
